In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from util.TSB_AD.slidingWindows import find_length #,plotFig, printResult
from sklearn.preprocessing import MinMaxScaler
import os
import stumpy
from util.TranAD_base import *
# from util.util_normal import plotFigKL, find_length, getResult, plotFigBeta
from util.util_overlap import *
from util.load_data import get_data, list_chunk, add_drift

from tqdm.notebook import tqdm

### Options of load_data_path
- filename: filepath+filename of .csv file
- data_type: each option has limited coverage for next options (character, val, width) 
    - 'None': no drift
    - 'abrupt': abrupt drift
        - (character)
            - 'mean': add 'val' to the sequences between {start-to-end} data (val=constant)
            - 'mean_flip': flip the {start-to-end} data, and add 'val' into them 
            - 'freq': resampling the {start-to-end} data with ratio of 'val' (e.g. val=1: no change, val=0.5: subsampling of half)
            - 'freq_flip': flip the {start-to-end} data, and resampling them with ratio of 'val'
    - 'inc': incremental drift
        - (character)
            - 'mean': {start-to-start+width} and {end-to-end+width} are transition period with adding slope (0-to-val). {start+width-to-end} is same as abrupt-mean drift
            - 'freq': {start-to-start+width} and {end-to-end+width} are transition period with (1+rval)/2 subsampling. {start+width-to-end} is same as abrupt-freq drift
    - 'grad': gradual drift
        - (character)
            - 'mean_flip': {start-to-start+width} and {end-to-end+width} are transition period with randomly selected subsequence with length slidingWindow. The random ratio is incrementally changed

In [ ]:
def load_data_path(filename, data_type, character, val, start, end, width, max_len):
    data_org = pd.read_csv(filename, header=None).to_numpy()
    if max_len == None:
        max_len = len(data_org)
    data = data_org[:max_len,0].astype(float)
    label=data_org[:max_len,1]

    win = find_length(data)

    data, label, label_cd = add_drift(data, label, data_type, start, end, character, val, win, width)
    return data, label, label_cd, win

### Add data files selectively...
- Due to the limited memory (and number of files)

In [ ]:
filepath = './data/benchmark/ECG/'
# filepath = './data/benchmark/Dodgers/'
# filepath = './data/benchmark/IOPS/'
# filepath = './data/benchmark/SensorScope/'
ext = '.out'
start = 'MBA_ECG8'
# start = '101'
# start = 'KPI'

tr_rate = 0.3
size_L = 10

filenames = []
for filename in os.listdir(filepath):
    if filename.endswith(ext): 
        if filename.startswith(start):
            # print(filename[26:])
            filenames.append(filename)

df_results = pd.DataFrame()

filenames = filenames[:1]
params = [
    # ['None', 'mean', 2, 0.4, 0.5, 0.1],
    # ['abrupt', 'mean', 30, 0.4, 0.5, 0.1],
    # ['inc', 'mean', 2, 0.4, 0.5, 0.1],
    # ['abrupt', 'mean_flip', 0, 0.6, 0.9, 0.1],
    ['grad', 'mean_flip', 1, 0.6, 0.9, 0.2],
    # ['abrupt', 'freq', 0.8, 0.6, 0.9, 0.1]
]
i=0
file = filenames[0]
data, label, label_cd, slidingWindow = load_data_path(filepath+file, params[i][0], params[i][1], params[i][2], start=params[i][3], end=params[i][4], width=params[i][5], max_len = 50000)